In [1]:
from owlready2 import *
#from owlready.editor import *
#from owlready.instance_editor import *
import types
from pprint import pprint
import editobj3
from bs4 import BeautifulSoup

In [2]:
onto = get_ontology("http://test.org/onto.owl")

In [3]:
def retrieve_info_from_page(asin):

    item_characteristics_dict = {}

    page = open('./pages/'+asin, 'r')
    soup = BeautifulSoup(page.read(), 'html.parser')
    tab = soup.find('div', {'id': 'HLCXComparisonWidget_feature_div'})

    # PRODUCT TITLE
    item_characteristics_dict['name'] = soup.find('span', {'id':'productTitle'}).text.strip().replace(' ', '_').replace('"', "").replace("'", "").replace("-", "").replace(",", "").replace(".", "")
    
    # PRODUCT ASIN
    item_characteristics_dict['asin'] = asin
    
    # PRODUCT CLASSES
    item_characteristics_dict['classes'] = []
    tmp = soup.find('div', {'id':'wayfinding-breadcrumbs_feature_div'})
    for i in tmp.findAll('a'):
        item_characteristics_dict['classes'].append(i.text.strip().replace(' ', '_').replace('&', 'and'))
        
    # PRODUCT BRAND
    if soup.find('a', {'id': 'bylineInfo'}) != None:
        item_charasteristics_dict = soup.find('a', {'id': 'bylineInfo'}).text.strip()
    # to finish

    bullets = soup.find('div', {'id':'feature-bullets'})
    #for span in bullets.findAll('span'):
        #print span.text.strip()

    try:
        for i, tr in enumerate(tab.findAll('tr')):
            # tr 0:
            # tr 1:
            # tr 2:
            # tr 3: customer rating
            if i<4: continue

            key = tr.findAll('td')[0].span.text
            if tr.findAll('td')[1].span != None: value = tr.findAll('td')[1].span.text
            else: value = tr.findAll('td')[1].a.text
            item_characteristics_dict[key.strip().lower()] = value.strip().lower()
    except: 
        print('no tr tab')
        
    # PROD DETAILS
    details_tab = soup.find('div', {'id':'prodDetails'})
    #print details_tab

    if soup.find('div', {'id':'productDescription'})!=None: 
        description1 = soup.find('div', {'id':'productDescription'}).p.text
        #print(description1)

    #pprint(item_characteristics_dict, indent=4)
    
    return item_characteristics_dict

In [4]:
dict_list = []
for i, page in enumerate(os.listdir('./pages')):
    dict_list.append(retrieve_info_from_page(page))
    
#print(dict_list)

no tr tab


In [5]:
for i in onto.classes():
    print(i)

In [6]:
# CLASSES
for i in dict_list:
    with onto:
        if "classes" in i:
            if i["classes"][0] not in [j._name for j in onto.classes()]: 
                New_Class1 = types.new_class(i["classes"][0], (Thing,))

            if i["classes"][1] not in [j._name for j in onto.classes()]: 
                New_Class2 = types.new_class(i["classes"][1], (Thing,))
                New_Class2.is_a.append(next(j for j in onto.classes() if j.name == i["classes"][0]))

            if i["classes"][2] not in [j._name for j in onto.classes()]: 
                New_Class3 = types.new_class(i["classes"][2], (Thing,))
                New_Class3.is_a.append(next(j for j in onto.classes() if j.name == i["classes"][1]))

In [7]:
# THINGS
for i in dict_list:
    with onto:
        #if i["name"] not in [j._name for j in onto.classes()]:
        New_Class1 = types.new_class(i["name"], (Thing,))
        New_Class1.is_a.append(next(j for j in onto.classes() if j.name == i["classes"][2]))

        if "color" in i:
            has_color = types.new_class("has_color", (DataProperty,))
            has_color.range = [str]
            New_Class1.has_color.append(str(i["color"]))
            print(New_Class1.has_color)

['black']
['red']
['sapphire black']
['rose gold']
['silver']
['white']
['black']
['silver']
['black']
['2 packs']
['black & black']
['black']
['black']
['silver']
['black']
['black']
['white']
['black']


In [15]:
sync_reasoner()
for i in onto.classes():
    print(i.has_color)

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /home/rana/Envs/amazon3/lib/python3.5/site-packages/owlready2/hermit:/home/rana/Envs/amazon3/lib/python3.5/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmphy9hr1i_


[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['black']
[]
[]
[]
[]
['red']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['sapphire black']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['rose gold']
['silver']
[]
[]
[]
['white']
[]
[]
[]
['black']
['silver']
[]
[]
[]
[]
[]
[]
[]
['black']
[]
[]
[]
[]
[]
[]
[]
[]
['2 packs']
[]
['black & black']
[]
[]
['black']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['black']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['silver']
['black']
[]
[]
['black']
[]
[]
[]
[]
[]
['white']
[]
['black']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


* Owlready2 * HermiT took 0.9980196952819824 seconds
* Owlready * Reparenting onto.Samsung_SMG900V__Galaxy_S5__16GB_Android_Smartphone_Verizon___Black_(Certified_Refurbished): {owl.Thing, onto.Unlocked_Cell_Phones} => {onto.Unlocked_Cell_Phones}
* Owlready * Reparenting onto.OnePlus_3_RAM_6GB+ROM_64GB_4G_FDDLTE_55_inch_Android_60_Smart_Phone_Qualcomm_Snapdragon_820_Quad_Core_2x22GHz_+_2x16GHz_80MP+160MP_(Grey): {owl.Thing, onto.Unlocked_Cell_Phones} => {onto.Unlocked_Cell_Phones}
* Owlready * Reparenting onto.OnePlus_3T_A3003_Unlocked_Smartphone_55Inch_128GB_DualSIM_4G_LTE__International_Version_(Gunmetal): {owl.Thing, onto.Unlocked_Cell_Phones} => {onto.Unlocked_Cell_Phones}
* Owlready * Reparenting onto.Unlocked_Cell_Phones_DOOGEE_X10_Dual_Sim_Smartphones_With_50_IPS_Display__Android_60__8GB_ROM__2MP+5MP_Dual_Camera__3360mAh_Battery__GSM_Unlocked_Phone_International__Black(no_ads)_…: {owl.Thing, onto.Unlocked_Cell_Phones} => {onto.Unlocked_Cell_Phones}
* Owlready * Reparenting onto.S

In [16]:
onto.OnePlus_One_A0001_4G_LTE_3GBRAM_64GBROM_55_Inch_International_Version__No_Warranty__Sandstone_Black.has_color

[]

In [22]:
onto.search( has_color = "*")

[]

In [ ]:
camera_set = set()
color_set = set()
for i in dict_list:
    if 'camera description' in i: camera_set.add(i['camera description'])
    if 'color' in i: color_set.add(i['color'])
    
pprint(color_set)

In [ ]:
Phone = types.new_class("Phone", (Thing,), kwds = { "ontology" : onto })
Camera = types.new_class("Camera", (Thing,), kwds = { "ontology" : onto })
Color = types.new_class("Color", (Thing,), kwds = { "ontology" : onto })
has_camera = types.new_class("has_camera", (Property,), kwds = { "ontology" : onto })
has_color = types.new_class("has_color", (Property,), kwds = { "ontology" : onto })
ANNOTATIONS[has_camera]["python_name"] = "has_camera"
ANNOTATIONS[has_color]["python_name"] = "has_color"

In [ ]:
for i in color_set:
    Color(i)

In [ ]:
for i in onto.Color._direct_instances:
    print(vars(i))

In [ ]:
for i in camera_set:
    Camera(i)

In [ ]:
for i in dict_list:
    which_color = ""
    if 'color' not in i: continue
    for j in onto.Color._direct_instances:
        if j._name == i['color']:
            which_color = j
    New_Class = types.new_class(i["asin"][:-5], (Phone,), kwds = { "ontology" : onto })
    New_Class.equivalent_to = [Phone & restriction(has_color, VALUE, j)]

In [ ]:
for i in dict_list:
    

In [ ]:
pprint(onto.instances)

In [ ]:
class Phone(Thing):
    ontology = onto
    
class Camera(Thing):
    ontology = onto
    
class has_camera(Property):
    ontology = onto
    domain = [Phone]
    range = [Camera]
ANNOTATIONS[has_camera]["python_name"] = "has_camera"

camera_8mp = Camera("camera_8mp")
camera_16mp = Camera("camera_16mp")

class galaxy_s8 (Phone):
    ontology = onto
    equivalent_to = [Phone & restriction(has_camera, VALUE, camera_16mp)]
    
    def print_name(self):
        print("hey there")

In [ ]:
pprint(vars(onto.galaxy_s8))

In [ ]:
query = Phone(has_color = [onto.black])
query.closed_world()
onto.sync_reasoner()

In [ ]:
print(vars(query))

In [ ]:
for ontology in onto.indirectly_imported_ontologies(): configure_editobj_from_ontology(ontology)

editobj3.GUI = "Qt" # "Qt" or "HTML"

if editobj3.GUI == "Qt": # Required by Qt
    import sys, PyQt5.QtWidgets as qtwidgets
    if qtwidgets.QApplication.startingUp(): qtwidgets.app = qtwidgets.QApplication(sys.argv)

editor = OntologyInstanceEditor()
editor.set_ontology(onto, edited_classes = onto.classes)
editor.main()